In [1]:
!pip3 install whisper-timestamped # meen hnaaaa ???


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.4 MB/s eta 0:00:001.3 MB/s eta 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.0/795.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 1.8 MB/s eta 0:00:000:00:010:00:01:03


In [1]:
import librosa
import numpy as np
import whisper_timestamped as whisper

import os


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



## Reading Libri-Speech

In [2]:
import pandas as pd


In [3]:
def load_audio_and_targets(libri_speech_dir):
  libri_dict = {}
  for root, dirs, files in os.walk(libri_speech_dir):
    for file in files:
      if file.endswith('.txt'):
        target_path = os.path.join(root, file)
        with open(target_path, 'r') as f:
          targets =  f.readlines()
          for target in targets:
            target = target.split()
            file = os.path.join(root, target[0]+".mp3")
            target = " ".join(target[1:])
            libri_dict[file] = target
  return libri_dict

In [ ]:
libri_dict = load_audio_and_targets("/home/g7/Desktop/LibriSpeech/train-clean-100/")

## Searching by Baby Names

In [5]:
df = pd.read_csv('/home/g7/Desktop/baby-names.csv')
df.drop(['year', 'percent', 'sex'], inplace=True, axis=1)
df = df[df['name']!='Will']
df = df.head(250)

In [6]:
names_dict = {}
name_dict = {}
for i in range(len(df)):
  for key, val in libri_dict.items():
    if df.iloc[i, 0].lower() in val.lower().split():
      name_dict['target'] = val
      name_dict['name'] = df.iloc[i, 0]
      name_dict['start_time'] = 0
      name_dict['end_time'] = 0
      names_dict[key] = name_dict
      name_dict = {}

In [7]:
len(libri_dict)

28539

### Getting start and end times

In [8]:
def get_start_end_time(name_dict, model):
  key, val = name_dict
  audio = whisper.load_audio(key)
  result = whisper.transcribe(model, audio, language='en')
  for segment in result['segments']:
    for word in segment['words']:
      if word['text'].lower() == val['name'].lower():
        return (word['start'], word['end'])
  return (0, 0)

In [9]:
def inject_start_end(names_dict, model):
  for i,(key, val) in enumerate(names_dict.items()):
    start_time, end_time = get_start_end_time((key,val), model)
    val['start_time'] = start_time
    val['end_time'] = end_time
    print(f"{i+1}/{len(names_dict)}" , end='\r')
  return names_dict



### Loading webScraped Names and adding new targets

In [10]:
name_dirs = []
for file in os.listdir('/home/g7/names-dataset/audio'):
  print(file)
  if file.endswith('.mp3'):
    name_dirs.append(os.path.join('/home/g7/names-dataset/audio', file))

Oihan_0.mp3
Dand_0.mp3
Moira_1.mp3
Pino_5.mp3
Rana_12.mp3
Lata_0.mp3
Juul_1.mp3
Velvet_4.mp3
Felisa_1.mp3
Eda_3.mp3
Iwan_2.mp3
Zanna_2.mp3
Dai_2.mp3
Thierry_2.mp3
Robin_0.mp3
Alban_3.mp3
Ivana_0.mp3
Malika_0.mp3
Holden_3.mp3
Charlène_0.mp3
Ambrosia_8.mp3
Ararat_5.mp3
Annie_4.mp3
Yael_4.mp3
Natalia_3.mp3
Reid_2.mp3
Teodor_1.mp3
Pascal_3.mp3
Andreas_14.mp3
Gilgamesh_0.mp3
İslam_18.mp3
Hilda_0.mp3
Grant_2.mp3
Margriet_0.mp3
Itala_0.mp3
Otakar_0.mp3
Rupa_0.mp3
Arianwen_0.mp3
Sibylle_2.mp3
Quetzalcoatl_1.mp3
Takumi_1.mp3
Amaterasu_1.mp3
Kerstin_4.mp3
Clarity_3.mp3
Neeltje_0.mp3
Aurelia_5.mp3
Goliath_4.mp3
Izar_3.mp3
Klára_3.mp3
Aladdin_4.mp3
Magnolia_2.mp3
Rhiannon_6.mp3
Magnolia_14.mp3
Hermine_4.mp3
Vasa_1.mp3
Yahweh_2.mp3
Tybalt_1.mp3
Skadi_0.mp3
Māui_0.mp3
Gloria_0.mp3
Latife_0.mp3
Hubert_11.mp3
Roman_17.mp3
Filadelfo_0.mp3
Anni_6.mp3
Amator_3.mp3
Eugenia_7.mp3
Valeria_9.mp3
Natacha_2.mp3
Sho_0.mp3
Cornel_2.mp3
Lone_1.mp3
Lohan_1.mp3
Ivone_2.mp3
Yanka_0.mp3
Ahmet_4.mp3
Romulus_3.mp3
İsla

In [11]:
import random
random.shuffle(name_dirs)
name_dirs = name_dirs[:100]

In [19]:
def add_new_target(names_dict, name_dirs, num_copies):

  for key, val in names_dict.items():
    copies = []
    random_indecies = random.sample(range(len(name_dirs)), num_copies)
    for index in random_indecies:
      name_dict = {}
      tagged_transcripts = {}
      # tagged_words = []#new
      name_dict['name_path'] = name_dirs[index]
      new_name = os.path.basename(name_dirs[index])
      new_name = new_name.split('.')[0]
      new_name = new_name.split('_')[0]
      # tagged_words.append((new_name, 'PER'))#new
      
      new_target = val['target'].split()
      name_index = new_target.index(val['name'].upper())
      new_target[name_index] = new_name.upper()
      new_target = " ".join(new_target)
      name_dict['target'] = new_target
      copies.append(name_dict)
    # tagged_transcripts[key] = tagged_words #new #check if key is the path

    names_dict[key]['copies'] = copies
  return names_dict


In [ ]:
new_names_dict , ground_truth_annotations= add_new_target(names_dict, name_dirs, 2)

In [ ]:
print(new_names_dict)

In [ ]:
# df = pd.read_csv('/home/g7/Desktop/baby-names.csv')
# df.drop(['year', 'percent', 'sex'], inplace=True, axis=1)
# df = df[df['name']!='Will']
# df = df.head(329)

In [3]:
#append names in df to new_names_dict
#new_df

### annotate ground truth for ner evluation 


In [ ]:
def tag_names_in_transcripts(new_names_dict, new_df):
    tagged_transcripts = {}
    for key, val in new_names_dict.items():
        words = val.lower().split()
        tagged_words = []  

        for i in range(len(df)):
            name = df.iloc[i, 0].lower()
            if name in words:
                tag = 'PER'
                tagged_words.append((name, tag))

        if tagged_words:
            tagged_transcripts[key] = tagged_words

    return tagged_transcripts

In [ ]:
ground_truth_annotations = tag_names_in_transcripts(new_names_dict, new_df)

In [14]:
model = whisper.load_model("tiny", device="cpu")
new_names_dict = inject_start_end(new_names_dict, model)

{'/home/g7/Desktop/LibriSpeech/train-clean-100/8580/287363/8580-287363-0031.mp3': [('Alberta', 'PER')]}


In [25]:
count = 0
for key, val in new_names_dict.items():
    if val["start_time"] == val["end_time"]:
        count+=1

print(count)

990


## Reading Audio Files

In [26]:
SAMPLING_RATE = 22050

In [27]:
# read audio files
def load_audio(audio_path):
  audio, _ = librosa.load(audio_path, sr = sampling_rate)
  return audio


In [28]:

def load_name(audio_file,sampling_rate, threshold=0.009):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=sampling_rate)

    # Find segments above the threshold
    non_silent_indices = np.where(abs(y) > threshold)[0]

    if len(non_silent_indices) > 0:
        # Determine start and end indices of non-silent audio
        start_index = non_silent_indices[0]
        end_index = non_silent_indices[-1]

        # Trim audio to non-silent regions
        trimmed_audio = y[start_index:end_index+1]

        return trimmed_audio
    else:
        # If no non-silent regions found, return None
        return y


## Processing the audios

In [29]:
# gets the ranges we remove from the sentence audio file
def get_concate_region(start_time, end_time, sampling_rate):
    start_index = int(start_time * sampling_rate)
    end_index = int(end_time * sampling_rate)
    return start_index, end_index

In [30]:
# pad audio left or right
def pad_audio(audio, left_pad, right_pad, mode="constant"):
    return np.pad(audio, (left_pad, right_pad), mode)

In [31]:
# padding the name file, first we get the start_index, that we pad left to, for padding right
# we set the desired dutaion for example 15 seconds, so that both audios are the exact same length
def prepare_audio_name(start_time, audio_name, sampling_rate, desired_duration):
  start_index, _ = get_concate_region(start_time, 0, sampling_rate)
  end_index = desired_duration*sampling_rate - (len(audio_name) + start_index)
  padded_audio = pad_audio(audio_name, start_index, end_index)
  return padded_audio

In [32]:
# first we pad right to the desired length, then get the start and end indecies
# for this we get the the desired duration and pad to it
# then we stretch or compress the segment we will be switching so that they are the same length
# for example if the name audio file was 2 seconds long and the segment we want to replace the name with in the sentece audio was 1 second
# we can stretch that segment to 2 seconds, with a stretch rate equal to 2
def prepare_scentence(start_time, end_time, audio_name, audio_scentence, sampling_rate, desired_duration):

  start_index, end_index = get_concate_region(start_time, end_time, sampling_rate)
  audio_name_len = len(audio_name)
  audio_scent_seg_len = end_index - start_index
  steretch_rate = audio_scent_seg_len/audio_name_len
  stretched_segment = audio_scentence[start_index:end_index]
  stretched_segment = librosa.effects.time_stretch(stretched_segment, rate=steretch_rate)
  new_end_index = start_index + len(stretched_segment)

  stretched_audio_scentece = np.concatenate((audio_scentence[:start_index], stretched_segment, audio_scentence[new_end_index:]))
  target_length = int(desired_duration * sampling_rate)

  audio_scentence_padded = pad_audio(stretched_audio_scentece, 0, (target_length- len(stretched_audio_scentece)))
  return audio_scentence_padded,start_index, new_end_index

# Adjusting pitch of name to match scentence's pitch

In [33]:
def detect_pitch(y, sr, t_begin, t_end):
  pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, fmin=75, fmax=1600)
  pitches_final = []
  print(pitches.shape)
  for t in range(t_begin, t_end):
    index = magnitudes[:, t].argmax()
    pitch = pitches[index, t]
    pitches_final.append(pitch)


  return pitches_final

In [34]:
def pitch_shift_audio(input_audio, target_audio, sampling_rate, start=0.54, stop=1.12):
    duration_seconds = 30  # Duration of the audio in seconds
    n_frames = 1292  # Number of frames (from the output shape)
    duration_samples = duration_seconds * sampling_rate
    hop_lengthh = duration_samples / (n_frames - 1)
    start = start / sampling_rate
    stop = stop / sampling_rate
    frame_index_begin = librosa.time_to_frames(start, sr=sampling_rate, hop_length=hop_lengthh)
    frame_index_end = librosa.time_to_frames(stop, sr=sampling_rate, hop_length=hop_lengthh)

    pitch_input = detect_pitch(input_audio, sampling_rate, frame_index_begin, frame_index_end)
    pitch_target = detect_pitch(target_audio , sampling_rate, frame_index_begin, frame_index_end)

    pitch_input = np.array(pitch_input)
    pitch_target = np.array(pitch_target)
    pitch_input = pitch_input[pitch_input!=0]
    pitch_target = pitch_target[pitch_target!=0]

    pitch_input = np.mean(pitch_input)
    pitch_target = np.mean(pitch_target)

    semitone_difference = round(12 * np.log2(pitch_target / pitch_input))
    print(semitone_difference)
    shifted_audio = librosa.effects.pitch_shift(input_audio, sr = sampling_rate, n_steps=semitone_difference)

    return shifted_audio

# Adjusting loudness of name to match scentece by normalization

In [35]:
def match_target_volume(input_audio, target_audio, start_time, end_time, sampling_rate):
    adjusted_audio = input_audio
    start_index, end_index = get_concate_region(start_time, end_time, sampling_rate)
    input_audio = input_audio[start_index:end_index]
    target_audio = target_audio[start_index:end_index]

    rms_input = np.sqrt(np.mean(input_audio**2))
    rms_target = np.sqrt(np.mean(target_audio**2))

    volume_ratio = rms_target / rms_input

    adjusted_audio[start_index:end_index] = input_audio * volume_ratio

    return adjusted_audio


In [ ]:
import matplotlib.pyplot as plt
import librosa.display

def visualize_audio_waves(audio, sampling_rate):
  plt.figure(figsize=(10, 4))
  librosa.display.waveshow(audio, sr=sampling_rate)
  plt.xlabel('Time (s)')
  plt.ylabel('Amplitude')
  plt.title('Audio Waveform')
  plt.show()

In [ ]:
visualize_audio_waves(audio_name, sampling_rate)


In [ ]:
visualize_audio_waves(audio_scentence, sampling_rate)

In [37]:
# truning the audio into mel spectrograms
def audio_to_spectrogram(audio_data, sample_rate, hop_length,  n_fft, n_mels, win_length):
  mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, hop_length=hop_length, n_fft=n_fft, n_mels=n_mels, win_length=win_length)
  mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
  return mel_spectrogram_db


## Concatenating Spectrograms

In [38]:
def concatenate_spectrograms(spectrogram_1, spectrogram_2, hop_length, sampling_rate, start_time, end_index, strength = 0.8):
  print(spectrogram_1.shape)
  print(spectrogram_2.shape)
  if spectrogram_1.shape != spectrogram_2.shape:
      raise ValueError("Spectrograms must have the same shape for concatenation")

  concatenated_spectrogram = np.zeros_like(spectrogram_1)
  start_index, _ = get_concate_region(start_time, end_time , sampling_rate)
  start_index = int(start_index/hop_length)

  end_index = int(end_index/hop_length)
  for i in range(spectrogram_1.shape[1]):  # Iterate over frequency bins on time axis so each slice is one slit containing a 1d matrix whish represents the frequancies in that time index

    if i > start_index and i < end_index:
      concatenated_spectrogram[:, i] = (1-strength)* spectrogram_1[:, i] + strength* spectrogram_2[:, i] # we can change the coefficients with experimentation
    else:
      concatenated_spectrogram[:, i] = spectrogram_1[:, i]

  return concatenated_spectrogram

In [39]:
n_fft = 2048
hop_length = 256
n_mels = 256
win_length = 1024
sampling_rate = 22050
duration = 30

In [40]:
def save_spectrogram(spectrogram, output_file):
    np.save(output_file, spectrogram)

In [44]:
input_path = []
target = []

input_path_pitch_shifted = []
i = 0
for key, val in new_names_dict.items():
  if(i<10):
    for copy in val['copies']:
      audio_name = load_name(copy['name_path'], sampling_rate)
      print(copy['name_path'])
      audio_scentence = load_audio(key)
      start_time = val['start_time']
      end_time = val['end_time']
      if start_time == 0 and end_time == 0:
        print("no start or end time")
        continue
      audio_scentence_prepared,start_index, end_index = prepare_scentence(start_time, end_time, audio_name, audio_scentence, sampling_rate, duration)

      audio_name_prepared = prepare_audio_name(start_time, audio_name, sampling_rate, duration)

      audio_name_prepared_normalized = match_target_volume(audio_name_prepared, audio_scentence_prepared, start_time, end_time, sampling_rate)
      try:
        audio_name_prepared_normalized_pitch_shifted = pitch_shift_audio(audio_name_prepared_normalized, audio_scentence_prepared, sampling_rate, start_index, end_index)
      except ValueError:
        print(ValueError)
        continue

      mel_spectrogram_db_name_normalized_pitch_shifted = audio_to_spectrogram(audio_name_prepared_normalized_pitch_shifted, sampling_rate, hop_length, n_fft, n_mels, win_length)

      mel_spectrogram_db_scentence = audio_to_spectrogram(audio_scentence_prepared, sampling_rate, hop_length, n_fft, n_mels, win_length)
      mel_spectrogram_db_name = audio_to_spectrogram(audio_name_prepared, sampling_rate, hop_length, n_fft, n_mels, win_length)

      concatenated_specs = concatenate_spectrograms(mel_spectrogram_db_scentence, mel_spectrogram_db_name, hop_length, sampling_rate, start_time,end_index, strength=1)
      concatenated_specs_normalized_pitch_shifted = concatenate_spectrograms(mel_spectrogram_db_scentence, mel_spectrogram_db_name_normalized_pitch_shifted, hop_length, sampling_rate, start_time,end_index, strength =0.99)

      save_path = os.path.basename(key).split('.')[0] + '_' + os.path.basename(copy['name_path']).split('.')[0]
      save_path_org = os.path.join('/home/g7/Desktop/names-dataset/specs_spectrogram', save_path + '.npy')

      save_path_pitch_shifted = os.path.join('/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted', save_path + '.npy')

      input_path.append(save_path_org)
      input_path_pitch_shifted.append(save_path_pitch_shifted)
      print(save_path_pitch_shifted)
      target.append(copy['target'])
      visualize_audio_spec(mel_spectrogram_db_name, sampling_rate, hop_length)
      visualize_audio_spec(mel_spectrogram_db_scentence, sampling_rate, hop_length)
      visualize_audio_spec(concatenated_specs, sampling_rate, hop_length)
      visualize_audio_spec(concatenated_specs_normalized_pitch_shifted, sampling_rate, hop_length)
      save_path_1 = 'home/g7/Desktop/' + str(i) + '_name.png'
      save_path_2 = 'home/g7/Desktop/' + str(i) + '_scentence.png'
      save_path_3 = 'home/g7/Desktop/' + str(i) + '_concatenated_specs.png'
      save_path_4 = 'home/g7/Desktop/' + str(i) + '_concatenated_specs_normalized_pitch_shifted.png'
      # save_spectrogram(mel_spectrogram_db_scentence, save_path_1)
      # save_spectrogram(mel_spectrogram_db_name, save_path2)
      # save_spectrogram(concatenated_specs, save_path_3)
      # save_spectrogram(concatenated_specs_normalized_pitch_shifted, save_path_4)
      #save_spectrogram(concatenated_specs, save_path_org)
      #save_spectrogram(concatenated_specs_normalized_pitch_shifted, save_path_pitch_shifted)
      i+=1
      print(i)

/home/g7/names-dataset/audio/Romuald_2.mp3
(1025, 1292)
(1025, 1292)
-17
(256, 2584)
(256, 2584)
(256, 2584)
(256, 2584)
/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted_TEST/4018-107312-0013_Romuald_2.npy
1
/home/g7/names-dataset/audio/Ris_8.mp3
(1025, 1292)
(1025, 1292)
-26
(256, 2584)
(256, 2584)
(256, 2584)
(256, 2584)
/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted_TEST/4018-107312-0013_Ris_8.npy
2
/home/g7/names-dataset/audio/Pelle_5.mp3
(1025, 1292)
(1025, 1292)
-1
(256, 2584)
(256, 2584)
(256, 2584)
(256, 2584)
/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted_TEST/332-128985-0079_Pelle_5.npy
3
/home/g7/names-dataset/audio/Quinto_13.mp3
(1025, 1292)
(1025, 1292)
18
(256, 2584)
(256, 2584)
(256, 2584)
(256, 2584)
/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted_TEST/332-128985-0079_Quinto_13.npy
4
/home/g7/names-dataset/audio/Mara_7.mp3
(1025, 1292)
(1025, 1292)
-12
(256, 2584)
(256, 2584)
(256, 2584)
(256, 2584)
/home/g7/D

In [48]:
target

["YOU SENT UP FOR SNICKS I HAVE KNOWN YOU MAN AND BOY ROMUALD HILL THESE TWENTY SUMMERS AND NEVER HEARD A WORD AGAINST YOU TILL YOU GOT INTO SHUFFLE AND SCREW'S MILL OH THEY ARE A BAD YARN JOHN",
 "YOU SENT UP FOR SNICKS I HAVE KNOWN YOU MAN AND BOY RIS HILL THESE TWENTY SUMMERS AND NEVER HEARD A WORD AGAINST YOU TILL YOU GOT INTO SHUFFLE AND SCREW'S MILL OH THEY ARE A BAD YARN JOHN",
 'TRIBUTOR AND A GREAT MANY OF THE PIRATES WERE KILLED OR TAKEN AND ROC THE BRAZILIAN HAD A TERRIBLE FALL THIS MOST MEMORABLE FALL OCCURRED IN THE ESTIMATION OF PELLE ESQUEMELING',
 'TRIBUTOR AND A GREAT MANY OF THE PIRATES WERE KILLED OR TAKEN AND ROC THE BRAZILIAN HAD A TERRIBLE FALL THIS MOST MEMORABLE FALL OCCURRED IN THE ESTIMATION OF QUINTO ESQUEMELING',
 'BUT WHICH ARE NECESSARY TO MAKE UP THE TRUE CHARACTER OF A PIRATE THE HISTORIAN MARA SEEMS TO HAVE BEEN VERY MUCH CUT UP BY THE MANNER IN WHICH HIS FAVORITE HERO HAD ROUNDED OFF HIS PIRATICAL CAREER',
 'BUT WHICH ARE NECESSARY TO MAKE UP THE TRUE 

In [45]:
def spectrogram_to_audio(spectrogram, output_audio_file,sampling_rate = 22050):
  power_spectrogram = librosa.db_to_power(spectrogram)
  audio_signal = librosa.feature.inverse.mel_to_audio(power_spectrogram, sr = sampling_rate, n_fft = n_fft, hop_length = hop_length, win_length=win_length)
  #visualize_audio_wave(audio_signal, sampling_rate)
  sf.write(output_audio_file, audio_signal, sampling_rate)

In [47]:
import soundfile as sf
for i in range(len(input_path)):
    spectrogram = np.load(input_path[i])
    spectrogram_pitch_shifted = np.load(input_path_pitch_shifted[i])
    spectrogram_to_audio(spectrogram, '/home/g7/Desktop/names-dataset/audio_TEST/'+ str(i) + '.wav')
    spectrogram_to_audio(spectrogram_pitch_shifted, '/home/g7/Desktop/names-dataset/audio_shifted_TEST/' + str(i) + '.wav')

In [ ]:
def save_spec_image(spec, image_path):
  plt.figure(figsize=(10, 4))
  librosa.display.specshow(spec, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='mel')
  plt.colorbar(format='%+2.0f dB')
  plt.title('Mel Spectrogram')
  plt.xlabel('Time (s)')
  plt.ylabel('Frequency (Hz)')
  plt.savefig(image_path)


In [ ]:
!pip install --upgrade matplotlib
import matplotlib.pyplot as plt

def visualize_audio_wave(audio_data, sample_rate):
  plt.figure(figsize=(10, 4))
  librosa.display.waveshow(audio_data, sr=sample_rate)
  plt.title('Waveform')
  plt.xlabel('Time (s)')
  plt.ylabel('Amplitude')
  plt.show()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 1.6 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.4
    Uninstalling matplotlib-3.8.4:
      Successfully uninstalled matplotlib-3.8.4


In [ ]:
import pandas as pd
df = pd.DataFrame({'input': input_path, 'target':target })
df.to_csv('/home/g7/Desktop/names-dataset/specs_spectrogram/data.csv', index=False)

df = pd.DataFrame({'input': input_path_pitch_shifted, 'target':target })
df.to_csv('/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted/data.csv', index=False)


In [ ]:
def visualize_audio_spec(mel_spectrogram_db, sample_rate, hop_length):
  plt.figure(figsize=(10, 4))
  librosa.display.specshow(mel_spectrogram_db, sr=sample_rate, hop_length=hop_length, x_axis='time', y_axis='mel')
  plt.colorbar(format='%+2.0f dB')
  plt.title('Mel Spectrogram')
  plt.xlabel('Time (s)')
  plt.ylabel('Frequency (Hz)')
  plt.show()

In [ ]:
import soundfile as sf
df_specs = pd.read_csv('/home/g7/Desktop/names-dataset/specs_spectrogram/data.csv')
df_specs_pitch_shifted = pd.read_csv('/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted/data.csv')
print(df_specs.iloc[0]['input'])
print(df_specs_pitch_shifted.iloc[0]['input'])
path_spec = df_specs.iloc[0]['input']
path_spec_pitch_shifted = df_specs_pitch_shifted.iloc[0]['input']

# Load the spectrogram data
spectrogram = np.load(path_spec)
spectrogram_pitch_shifted = np.load(path_spec_pitch_shifted)
spectrogram_to_audio(spectrogram, '/home/g7/Desktop/names-dataset/spectrogram.wav')
spectrogram_to_audio(spectrogram_pitch_shifted, '/home/g7/Desktop/names-dataset/spectrogram_pitch_shifted.wav')

/home/g7/Desktop/names-dataset/specs_spectrogram/4018-107312-0013_Sallie_0.npy
/home/g7/Desktop/names-dataset/specs_spectrogram_pitch_shifted/4018-107312-0013_Sallie_0.npy
